In [ ]:
import pandas as pd
import requests
import re
from decimal import Decimal

EXCEL_INPUT = "../Account.xlsx"
EXCEL_OUTPUT = "cleaned_transactions.xlsx"
API_URL = "http://localhost:8000/transactions/create/"  # change to match your server
USER_ID = 1  # update if needed

# === STEP 1: Parse & Clean Excel ===

# Load Excel
df = pd.read_excel(EXCEL_INPUT, sheet_name="Compte")
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True, errors="coerce")
df["Description"] = df["Description"].fillna("").astype(str)
df["Produit"] = df["Produit"].fillna("")
df["Mouvements"] = pd.to_numeric(df["Unnamed: 8"], errors="coerce")

# Helpers
def detect_type(desc):
    desc = desc.lower()
    if "dépôt" in desc:
        return "deposit"
    if any(keyword in desc for keyword in ["tob", "impôt", "frais"]):
        print(desc)
        return "fee"
    if "dividende" in desc:
        return "dividend"
    if "achat" in desc:
        return "buy"
    if "vente" in desc:
        return "sell"
    return None

def extract_shares(desc):
    match = re.search(r"\b(?:Achat|Vente)\s+([\d.,]+)", desc)
    if match:
        return float(match.group(1).replace(",", "."))
    return None

# Find currency conversion rows
debits = df[df["Description"].str.contains("Opération de change - Débit", case=False)].copy()
credits = df[df["Description"].str.contains("Opération de change - Crédit", case=False)].copy()

debit_map = debits.groupby("Date")["Mouvements"].apply(list).to_dict()
credit_map = credits.groupby("Date")["Mouvements"].apply(list).to_dict()

debit_used = {}
credit_used = {}

# Main filter
df["type"] = df["Description"].apply(detect_type)
df["shares"] = df["Description"].apply(extract_shares)

relevant_df = df[df["type"].notnull()].copy()

# Parse rows
rows = []
for _, row in relevant_df.iterrows():
    tx_type = row["type"]
    date = row["Date"]
    description = row["Description"]
    original_amount = row["Mouvements"]
    amount = original_amount
    ticker = row["Produit"] if tx_type in ["buy", "sell", "dividend"] else None
    shares = row["shares"]
    metadata = {}

    # Handle currency conversion for foreign BUY/SELL
    if tx_type == "buy" and "eur" not in description.lower():
        used = debit_used.get(date, 0)
        debit_list = debit_map.get(date, [])
        if used < len(debit_list):
            amount = debit_list[used]
            debit_used[date] = used + 1
            metadata = {"converted_from": float(original_amount), "conversion_method": "Opération de change - Débit"}
    elif tx_type == "sell" and "eur" not in description.lower():
        used = credit_used.get(date, 0)
        credit_list = credit_map.get(date, [])
        if used < len(credit_list):
            amount = credit_list[used]
            credit_used[date] = used + 1
            metadata = {"converted_from": float(original_amount), "conversion_method": "Opération de change - Crédit"}

    rows.append({
        "type": tx_type,
        "date": date.strftime("%Y-%m-%d"),
        "amount": float(Decimal(amount).quantize(Decimal("0.01"))),
        "ticker": ticker,
        "shares": float(shares) if shares else None,
        "metadata": metadata,
        "user_id": USER_ID
    })

# Save cleaned transactions to Excel
cleaned_df = pd.DataFrame(rows)
# cleaned_df.to_excel(EXCEL_OUTPUT, index=False)
print(f"✅ Cleaned transactions saved to {EXCEL_OUTPUT}")


impôts sur dividende
impôts sur dividende
impôts sur dividende
impôts sur dividende
impôts sur dividende
impôts sur dividende
impôts sur dividende
taxe sur les opérations boursières belge (tob)
frais degiro de courtage et/ou de parties tierces
frais de connexion aux places boursières 2025 (new york stock exchange - nsy)
frais de connexion aux places boursières 2025 (nasdaq - ndq)
frais de connexion aux places boursières 2025 (xetra - xet)
impôts sur dividende
impôts sur dividende
impôts sur dividende
impôts sur dividende
taxe sur les opérations boursières belge (tob)
frais degiro de courtage et/ou de parties tierces
impôts sur dividende
taxe sur les opérations boursières belge (tob)
frais degiro de courtage et/ou de parties tierces
impôts sur dividende
impôts sur dividende
impôts sur dividende
impôts sur dividende
impôts sur dividende
taxe sur les opérations boursières belge (tob)
frais degiro de courtage et/ou de parties tierces
taxe sur les opérations boursières belge (tob)
frais deg

In [4]:

# === STEP 2: Load Cleaned File & POST to Django ===

cleaned_df = pd.read_excel(EXCEL_OUTPUT)

for _, row in cleaned_df.iterrows():
    payload = {
        "type": row["type"],
        "amount": float(row["amount"]),
        "date": row["date"],
        "user_id": int(row["user_id"]),
        "ticker": row["ticker"] if pd.notna(row["ticker"]) else None,
        "shares": float(row["shares"]) if pd.notna(row["shares"]) else None,
        "metadata": eval(row["metadata"]) if isinstance(row["metadata"], str) else row["metadata"],
    }

    print(f"📤 Sending: {payload}")
    try:
        response = requests.post(API_URL, json=payload)
        print(f"✅ {response.status_code}: {response.json()}")
    except Exception as e:
        print(f"❌ Error posting transaction: {e}")


[{'type': 'deposit',
  'amount': nan,
  'date': Timestamp('2025-05-05 00:00:00'),
  'user_id': 1,
  'ticker': None,
  'shares': None,
  'metadata': {}},
 {'type': 'dividend',
  'amount': nan,
  'date': Timestamp('2025-01-05 00:00:00'),
  'user_id': 1,
  'ticker': 'RITHM CAPITAL CORP',
  'shares': None,
  'metadata': {}},
 {'type': 'dividend',
  'amount': nan,
  'date': Timestamp('2025-01-05 00:00:00'),
  'user_id': 1,
  'ticker': 'RITHM CAPITAL CORP',
  'shares': None,
  'metadata': {}},
 {'type': 'dividend',
  'amount': nan,
  'date': NaT,
  'user_id': 1,
  'ticker': 'SALESFORCE.COM',
  'shares': None,
  'metadata': {}},
 {'type': 'dividend',
  'amount': nan,
  'date': NaT,
  'user_id': 1,
  'ticker': 'SALESFORCE.COM',
  'shares': None,
  'metadata': {}},
 {'type': 'dividend',
  'amount': nan,
  'date': NaT,
  'user_id': 1,
  'ticker': 'SALESFORCE.COM',
  'shares': None,
  'metadata': {}},
 {'type': 'dividend',
  'amount': nan,
  'date': NaT,
  'user_id': 1,
  'ticker': 'SALESFORCE.CO

In [18]:
df.tail(10)

,Date,Heure,Date de,Produit,Code ISIN,Description,FX,Mouvements,Unnamed: 8,Solde,Unnamed: 10,ID Ordre,type,shares
580,2022-02-25,18:43,25-02-2022,NVIDIA CORP,US67066G1040,Operation de change - Crédit,"1,1216",480.64,480.64,USD,"0,00",68aaf79d-fa81-4c8d-bb3b-019ed5c6b5f9,None,NaN
581,2022-02-25,18:43,25-02-2022,NVIDIA CORP,US67066G1040,Opération de change - Débit,NaN,-428.53,-428.53,EUR,"2 069,48",68aaf79d-fa81-4c8d-bb3b-019ed5c6b5f9,None,NaN
582,2022-02-25,18:43,25-02-2022,NVIDIA CORP,US67066G1040,Taxe sur les Opérations Boursières belge (TOB),NaN,-1.50,-1.50,EUR,"2 498,01",68aaf79d-fa81-4c8d-bb3b-019ed5c6b5f9,fee,NaN
583,2022-02-25,18:43,25-02-2022,NVIDIA CORP,US67066G1040,Frais DEGIRO de courtage et/ou de parties tierces,NaN,-0.50,-0.50,EUR,"2 499,51",68aaf79d-fa81-4c8d-bb3b-019ed5c6b5f9,fee,NaN
584,2022-02-25,18:43,25-02-2022,NVIDIA CORP,US67066G1040,"Achat 2 NVIDIA Corp@240,32 USD (US67066G1040)",NaN,-480.64,-480.64,USD,"-480,64",68aaf79d-fa81-4c8d-bb3b-019ed5c6b5f9,buy,2.0
585,2022-02-18,14:40,18-02-2022,,NaN,Dépôt flatex,NaN,400.00,400.00,EUR,"2 500,01",NaN,deposit,NaN
586,2022-02-17,14:40,17-02-2022,,NaN,Dépôt flatex,NaN,NaN,2 100.00,EUR,"2 100,01",NaN,deposit,NaN
587,2022-01-28,10:17,27-01-2022,,NaN,Virement vers votre Compte Espèces à la flatex...,NaN,NaN,NaN,EUR,"0,01",NaN,None,NaN
588,2022-01-28,10:17,27-01-2022,FLATEX EURO BANKACCOUNT,NLFLATEXACNT,Degiro Cash Sweep Transfer,NaN,-0.01,-0.01,EUR,"0,00",NaN,None,NaN
589,2022-01-27,18:58,27-01-2022,,NaN,Versement de fonds,NaN,0.01,0.01,EUR,"0,01",NaN,None,NaN
